In [1]:
import requests
import json
from datetime import datetime
import pytz
import pandas as pd
import traceback

from liquer import *

HEADERS_FOR_JSON = {
    'accept': "application/json; charset=utf-8",
    'content-type': "application/json"
}

SEARCH_URL = "http://distribution.virk.dk/offentliggoerelser/_search"


def to_datetime(d):
    if isinstance(d, datetime):
        return d.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
    if d in (None, "NOW"):
        return datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'


NS = "virk"


@first_command(ns=NS)
def offentliggoerelser_raw(day=None, size=2999):
    """Get public register as JSON document.

    Arguments:
    day -- date string in YYYYMMDD format
    size -- size argument in the query
    """
    if day in (None, "TODAY"):
        d = datetime.utcnow()
    else:
        d = datetime.strptime(day, "%Y%m%d")
    dd = d.strftime("%Y-%m-%d")
    gt = f"{dd}T00:00:00.001Z"
    lt = f"{dd}T23:59:59.505Z"
    data = {
        "query": {
            "bool": {
                "must": [
                    {
                        "term": {
                            "dokumenter.dokumentMimeType": "application"
                        }
                    },
                    {
                        "term": {
                            "dokumenter.dokumentMimeType": "xml"
                        }
                    },
                    {
                        "range": {
                            "offentliggoerelsesTidspunkt": {
                                "gt": gt,
                                "lt": lt
                            }
                        }
                    }
                ],
                "must_not": [],
                "should": []
            }
        },
        "size": size
    }

    response = requests.post(SEARCH_URL, data=json.dumps(data),
                             headers=HEADERS_FOR_JSON)
    return response.json()


@command(ns=NS)
def register2df(data):
    """Convert JSON response from offentliggoerelser_raw to DataFrame"""
    df = pd.DataFrame(columns=["_id", "_index", "_score", "cvrNummer"])
    for r in data["hits"]["hits"]:
        d = dict(
            _id=r["_id"],
            _index=r["_index"],
            _score=r["_score"],
            cvrNummer=str(r["_source"]["cvrNummer"]),
            indlaesningsId=r["_source"]["indlaesningsId"],
            indlaesningsTidspunkt=r["_source"]["indlaesningsTidspunkt"],
            offentliggoerelsesTidspunkt=r["_source"]["offentliggoerelsesTidspunkt"],
            offentliggoerelsestype=r["_source"]["offentliggoerelsestype"],
            omgoerelse=r["_source"]["omgoerelse"],
            regNummer=r["_source"]["regNummer"],
            regnskabsperiode_startDato=r["_source"]["regnskab"]["regnskabsperiode"]["startDato"],
            regnskabsperiode_slutDato=r["_source"]["regnskab"]["regnskabsperiode"]["slutDato"],
            sagsNummer=r["_source"]["sagsNummer"],
            sidstOpdateret=r["_source"]["sidstOpdateret"],
        )
        for doc in r["_source"]["dokumenter"]:
            t = {"application/xhtml+xml": "html", "application/pdf": "pdf",
                 "application/xml": "xml"}.get(doc["dokumentMimeType"])
            if t is not None:
                if doc["dokumentType"] == "AARSRAPPORT":
                    d[f"AARSRAPPORT_{t}"] = doc["dokumentUrl"]
        df = df.append(d, ignore_index=True)
    return df


@first_command(ns=NS)
def register(day="TODAY", size=2999):
    """Get public register as dataframe.
    Uses offentliggoerelser_raw to fetch the data and register2df to convert the data afterwards.

    Arguments:
    day -- date string in YYYYMMDD format
    size -- size argument in the query
    """
    return evaluate(f"ns-{NS}/offentliggoerelser_raw-{day}-{size}/register2df").get()


@first_command(ns=NS)
def cvr(cvr, ext="xml", day="TODAY", size=2999):
    """
    Get the document for a specified CVR number.
    Document has to be found in an index (obtained by the 'index' function taking day and size parameters).
    Multiple formats of document are supported by VIRK, format is specified by 'ext' (xml by default).
    """
    df = register(day=day, size=size)
    url = list(df.loc[df.cvrNummer.map(str) == str(cvr), f"AARSRAPPORT_{ext}"])
    if len(url):
        try:
            return requests.get(url[0]).text
        except:
            import traceback
            traceback.print_exc()
            return ""
    else:
        return ""


@command(ns=NS, volatile=True)
def tojson(xml):
    """Convert XML document with financial statements to JSON"""
    from xmljson import yahoo
    from lxml import etree
    try:
        root = etree.fromstring(xml.encode("utf-8"))
    except:
        return {}
    for elem in root.getiterator():
        try:
            tag = etree.QName(elem.tag)
        except:
            traceback.print_exc()
            continue
        elem.tag = tag.localname
        d = {}
        for key, value in elem.attrib.items():
            nkey = etree.QName(key).localname
            d[nkey] = value
            del elem.attrib[key]
        elem.attrib.update(d)

    d = yahoo.data(root)
    return d["xbrl"]


@command(ns=NS, volatile=True)
def json2df(doc, keep_multiline_values=False, init=None):
    """Convert JSON document with financial statements to DataFrame

    Arguments:
    keep_multiline_values -- if True, arguments with multiline strings are kept
    init -- parameter for internal use
    """
    if not isinstance(init, dict):
        init = {}
    df = pd.DataFrame(columns=["entity", "start_date", "end_date", "context"])
    context = {c["id"]: c for c in doc.get("context", [])}
    cdates = []
    for c in doc.get("context", []):
        period = c.get("period", {})
        cid = c["id"]
        if "instant" in period:
            cdates.append((period["instant"], period["instant"], cid))
        elif "startDate" in period and "endDate" in period:
            cdates.append((period["startDate"], period["endDate"], cid))
        else:
            raise Exception("Context without known dates")

    cdates = sorted(cdates)

    cols = sorted(key for key in doc.keys() if key not in ["context", "unit"])
    for start, end, identifier in cdates:
        c = context[identifier]
        entity = c["entity"]["identifier"]["content"]
        d = dict(init, start_date=start, end_date=end,
                 entity=entity, context=identifier)
        for key in cols:
            keydata = doc[key]
            if isinstance(keydata, list):
                row = [r for r in keydata if r["contextRef"] == identifier]
#                if len(row) > 1:
#                    print(
#                        f"Warning: multiple context entries for {entity} {key}, context:{identifier}")
                if len(row):
                    keydata = row[0]
                else:
                    keydata = None
            if keydata is not None:
                if isinstance(keydata, dict) and "content" in keydata:
                    use = keep_multiline_values or "\n" not in keydata["content"]
                    if use:
                        value = keydata["content"]
                        try:
                            value = float(value)
                            if int(value) == value:
                                value = int(value)
                        except:
                            pass
                        d[key] = value
        df = df.append(d, ignore_index=True)
    return df


@first_command(ns=NS)
def cvrdf(cvr, keep_multiline_values=False, day="TODAY", size=2999):
    """
    Get the document for a specified CVR number as DataFrame.
    Document has to be found in an index (obtained by the 'index' function taking day and size parameters).
    If keep_multiline_values is True, arguments with multiline strings are kept, otherwise they are removed (default).
    """
    register_df = register(day=day, size=size)
    register_df = register_df.loc[register_df.cvrNummer.map(
        str) == str(cvr), :]
    df = pd.DataFrame()
    for index, row in register_df.iterrows():
        cvr = str(row.cvrNummer)
        doc = evaluate(f"ns-{NS}/cvr-{cvr}-xml-{day}-{size}/tojson").get()
        cvr_df = json2df(
            doc, keep_multiline_values=keep_multiline_values, init=dict(row))
        df = df.append(cvr_df, ignore_index=True)
    return df

In [2]:
import liquer.ext.basic
import liquer.ext.lq_pandas
from datetime import datetime
from tqdm import tqdm_notebook
import warnings, sys, os
from liquer.cache import FileCache, set_cache
set_cache(FileCache("cache"))



In [3]:
def find_common_attributes(start_date, end_date):
    # Define the dates we are interested in
    dates = [date.strftime('%Y%m%d') for date in pd.date_range(start=start_date, end=end_date)] 

    attributes = []
    for date in tqdm_notebook(dates): # Loop over the dates and get the cvr of each update for that day
        register_df2 = register(date)
        cvrs = register_df2.cvrNummer  
        print(f'Number of cvr numbers in current iteration: {len(cvrs)}')
        for cvr_num in tqdm_notebook(cvrs):             # Find the attributes of that specific company and store it
            attributes.append(set(cvrdf(str(cvr_num), day=date).columns))

    return set.intersection(*attributes)

In [4]:
def find_all_attributes(start_date, end_date):
    # Define the dates we are interested in
    dates = [date.strftime('%Y%m%d') for date in pd.date_range(start=start_date, end=end_date)] 

    attributes = []
    for date in tqdm_notebook(dates): # Loop over the dates and get the cvr of each update for that day
        register_df2 = register(date)
        cvrs = register_df2.cvrNummer  
        print(f'Number of cvr numbers in current iteration: {len(cvrs)}')
        for cvr_num in tqdm_notebook(cvrs):             # Find the attributes of that specific company and store it
            attributes.append(set(cvrdf(str(cvr_num), day=date).columns))

    return set.union(*attributes)

In [5]:
def download_files(start_date, end_date, found_cvrs=set()):
    # Define the dates we are interested in
    dates = [date.strftime('%Y%m%d') for date in pd.date_range(start=start_date, end=end_date)] 
    skipped_cvrs = []
    cvr_set = found_cvrs
    print(type(cvr_set))
    for date in tqdm_notebook(dates): # Loop over the dates and get the cvr of each update for that day
        register_df2 = register(date)
        cvrs = register_df2.cvrNummer    
        print(f'Number of cvr numbers in current iteration: {len(cvrs)}')
        for cvr_num in tqdm_notebook(cvrs):             # Find the attributes of that specific company and store it
            if cvr_num in cvr_set:
                continue
            else:
                try:
                    cvrdf(str(cvr_num), day=date).to_csv(path_or_buf='New_data/' + str(cvr_num) + '.csv')
                except:
                    skipped_cvrs.append(cvr_num)
                    continue
        cvr_set.update(set(cvrs))
    return skipped_cvrs

In [6]:
start_date = '20190101'
end_date = '20190101'
common_attr = find_common_attributes(start_date, end_date)

Number of cvr numbers in current iteration: 159


Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cy

ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", l

  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx",

  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, i

...

  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx",

  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, i

Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cy

ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", l

  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx",

In [10]:
print(common_attr)
len(common_attr)

{'offentliggoerelsestype', 'NameOfReportingEntity', 'regnskabsperiode_slutDato', 'TypeOfAuditorAssistance', 'offentliggoerelsesTidspunkt', 'AARSRAPPORT_pdf', 'sidstOpdateret', 'cvrNummer', 'LiabilitiesAndEquity', '_index', 'context', 'indlaesningsTidspunkt', 'ProfitLoss', 'indlaesningsId', 'entity', 'regNummer', 'end_date', '_id', 'ClassOfReportingEntity', 'AddressOfSubmittingEnterpriseStreetAndNumber', 'AARSRAPPORT_xml', 'sagsNummer', 'omgoerelse', 'ReportingPeriodEndDate', 'InformationOnTypeOfSubmittedReport', 'AddressOfSubmittingEnterprisePostcodeAndTown', 'ReportingPeriodStartDate', '_score', 'start_date', 'Equity', 'IdentificationNumberCvrOfReportingEntity', 'AARSRAPPORT_html', 'regnskabsperiode_startDato', 'NameAndSurnameOfChairmanOfGeneralMeeting', 'Assets'}


35

In [9]:
all_attr = find_all_attributes(start_date, end_date)
len(all_attr)

Number of cvr numbers in current iteration: 159


Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cy

ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", l

  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx",

  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, i

...

  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, i

Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cy

ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", l

  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx", line 1835, in lxml.etree.QName.__init__
  File "src/lxml/apihelpers.pxi", line 1680, in lxml.etree._tagValidOrRaise
ValueError: Invalid tag name '<cyfunction Comment at 0x000001DB73381C80>'
Traceback (most recent call last):
  File "<ipython-input-1-d648b050eb9b>", line 152, in tojson
    tag = etree.QName(elem.tag)
  File "src/lxml/etree.pyx",

452

In [7]:
import os
current_cvrs = set()
#current_cvrs = set([int(file.split('.')[0]) for file in os.listdir("Data/")])
#current_cvrs.update([int(file.split('.')[0]) for file in os.listdir("Valid_Data/")])
#current_cvrs.update([int(file.split('.')[0]) for file in os.listdir("New_Data/")])
skipped_cvrs = download_files(start_date='20180101', end_date='20181231', found_cvrs=current_cvrs)

<class 'set'>


Number of cvr numbers in current iteration: 53


Number of cvr numbers in current iteration: 314


Number of cvr numbers in current iteration: 333


Number of cvr numbers in current iteration: 425


...

Number of cvr numbers in current iteration: 758


Number of cvr numbers in current iteration: 347


Number of cvr numbers in current iteration: 0


Number of cvr numbers in current iteration: 0


Number of cvr numbers in current iteration: 595


In [91]:
# Function to use later for aggregating our data for each CVR file. Also able to handle strings, return NaN if 
# different strings are present in the column
import numpy as np
from pandas.api.types import is_numeric_dtype

def mean_str(col):
    if is_numeric_dtype(col):
        return col.mean()
    else:
        return tuple(col.unique()) if col.nunique() == 1 else np.NaN

common_attr
#print(common_attr)
#useful_common_att = ['Assets','Equity','_score','ProfitLoss','LiabilitiesAndEquity']
#useful_all_att = ['Assets','Equity','_score','ProfitLoss','LiabilitiesAndEquity','Revenue','Dividend(Paid)',
                   'NumberOfEmployees','AverageNumberOfEmployees']

In [119]:
all_cvrs_df = pd.DataFrame()

for file in tqdm_notebook(os.listdir('Valid_Data/')):
    # Get the CVR number 
    cvr_num = file.split('.')[0]
    # Load the company CSV file into a pandas DF
    company_df = pd.read_csv('Valid_Data/' + file).drop(columns = 'Unnamed: 0')
    
    # Change the start_date column into only containing years for easier aggregation
    for idx in range(0, len(company_df)):
        company_df.at[idx, 'start_date'] = company_df.at[idx, 'start_date'][0:4]
    
    # Aggregate the rows by the 
    company_df = company_df.groupby('start_date').agg(mean_str)
    
    # Make a new column called target which includes next years revenue 
    company_df['Target'] = company_df['Revenue'].shift(-1)
     
    company_df = company_df[~company_df.Target.isna()]    
    
    all_cvrs_df = pd.concat([company_df, all_cvrs_df], ignore_index=True, axis = 0)
    

C:\Users\Bruger\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [121]:
all_cvrs_df.to_pickle('all_data.pkl')